# データサイエンス特論　プログラム課題　第2回

情報知能工学専攻　音声言語処理研究室

M203319 M1 木内貴浩 

以下の課題を実施し、実行したプログラムと実行結果(プログラムはPython, Jupyter notebook(拡張子.ipynb, .html)等でログをまとめるのも可、実行結果を MS Word(PDF)等でま とめることも可) を、ZIP でまとめ、MoodleLMS にアップロードすること。期限は【8 月 8 日(月)】の深夜 23:59 分までとする。(プログラム課題はこれで終わりです)


UCI(カリフォルニア大学アーバイン校)の機械学習用データレポジトリにある有名なロ イ タ ー の ニ ュ ー ス 記 事 ( Reuters-21578 ) https://archive.ics.uci.edu/ml/datasets/reuters- 21578+text+categorization+collection は、英語で書かれた1980 年当時のニュース記事の 集合 である。このデータは、機械学習の練習にしばしば利用されてきた。ニュース記事には カ テゴリー情報が 1 種類以上付与(マルチラベルが付与)されており、このカテゴリーラベ ルを予測するテキスト分類問題を考える。ただし、元のロイターニュース記事には、各種 のノイズ文書がある。そこで、55 カテゴリー、10,700 文書(7713 訓練文書, 2987 テスト文 書)に絞り込んだ 部分集合記事データを作成した。以下のサイトからダウンロードで き る 。 https://www.kde.cs.tut.ac.jp/~aono/data/ma_reuters.zip においてある。 以降、こ のデータをMA-Reuters と呼ぶこととする。


## Step 1 MA-Reutersを各自の作業できるPC等にダウンロードせよ。

In [6]:
!cd ../datas && wget https://www.kde.cs.tut.ac.jp/~aono/data/ma_reuters.zip

/Users/kinouchitakahiro/Documents/Personal


## Step 2 環境準備

MA-ReutersはPythonのNLTKパッケージのコーパスに従って作成してある。そこで、各自のPython の環境にNLTK がまだインストールされていない場合、NLTKをインストールせよ。

同時に https://www.nltk.org/data.html を参照してデータのダウンロードを最初に一回行い、適当な NLTK 用のデータフォルダーを作成しておくこと。た とえば Windows 環境下で、C:\nltk_data に作成したとすると、コーパス フォルダが、 C:\nltk_data\corpora 以下にできるので、(1)で入手したma_reuters.zip を corpora フォルダ にコピーしておくこと。

In [9]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

# Step 3 Scikit-learnのダウンロード

In [ ]:
! pip install scikit-learn

## ma_reuters の移動

In [12]:
! mv ../datas/ma_reuters.zip /Users/kinouchitakahiro/nltk_data/corpora

# 課題1 

https://www.kde.cs.tut.ac.jp/~aono/2022/DataScience/Reuters-Multi-Label-SVM-Example.html に ある Jupyter notebook でのPython コードを順番に実行し、多値クラス多値ラベル問題を 解決できることを確認せよ。ただし、このコード内でオレンジカテゴリー内の(最初の)文書 のプリントは、違うカテゴリー(オレンジカテゴリー以外)の文書に変更して実行するこ と。

## NLTKを使ったコーパスのロード

In [1]:
from nltk.corpus.util import LazyCorpusLoader
from nltk.corpus.reader import *

# コーパスのロード
ma_reuters = LazyCorpusLoader(
    'ma_reuters', CategorizedPlaintextCorpusReader, '(training|test).*',
    cat_file='cats.txt', encoding='ISO-8859-2')

# MA_Reutersのロード
documents = ma_reuters.fileids()
print( f"{str(len(documents))}  記事総数")

# 訓練とテストデータの文書IDの抽出
train_docs_id = [doc for doc in documents if doc.startswith("train")]
test_docs_id = [doc for doc in documents if doc.startswith("test")]

print( f"{str(len(train_docs_id))} 訓練データ")
print( f"{str(len(test_docs_id))} テストデータ")

# 訓練とテストデータの生データの抽出
train_docs = [ma_reuters.raw(doc_id) for doc_id in train_docs_id]
test_docs = [ma_reuters.raw(doc_id) for doc_id in test_docs_id]
 
# カテゴリーのリスト
categories = ma_reuters.categories()
num_categories = len(categories)

print( f"{num_categories} カテゴリー")
print( f"{categories}")

10700  記事総数
7713 訓練データ
2987 テストデータ
55 カテゴリー
['acq', 'alum', 'barley', 'bop', 'carcass', 'cocoa', 'coffee', 'copper', 'corn', 'cotton', 'cpi', 'crude', 'dlr', 'earn', 'fuel', 'gas', 'gnp', 'gold', 'grain', 'hog', 'housing', 'interest', 'ipi', 'iron-steel', 'jobs', 'lead', 'livestock', 'meal-feed', 'money-fx', 'money-supply', 'nat-gas', 'oilseed', 'orange', 'palm-oil', 'pet-chem', 'rapeseed', 'reserves', 'retail', 'rice', 'rubber', 'ship', 'silver', 'sorghum', 'soy-meal', 'soy-oil', 'soybean', 'strategic-metal', 'sugar', 'tin', 'trade', 'veg-oil', 'wheat', 'wpi', 'yen', 'zinc']


## shipのカテゴリーにある記事を表示

In [2]:
# 生の文書例（オレンジ・カテゴリー）
# Documents in a category
category_docs = ma_reuters.fileids("ship");
document_id = category_docs[0] # ship・カテゴリーの最初の文書

# 記事の中身を表示
print( f"{ma_reuters.raw(document_id)}")

AUSTRALIAN FOREIGN SHIP BAN ENDS BUT NSW PORTS HIT
  Tug crews in New South Wales (NSW),
  Victoria and Western Australia yesterday lifted their ban on
  foreign-flag ships carrying containers but NSW ports are still
  being disrupted by a separate dispute, shipping sources said.
      The ban, imposed a week ago over a pay claim, had prevented
  the movement in or out of port of nearly 20 vessels, they said.
      The pay dispute went before a hearing of the Arbitration
  Commission today.
      Meanwhile, disruption began today to cargo handling in the
  ports of Sydney, Newcastle and Port Kembla, they said.
      The industrial action at the NSW ports is part of the week
  of action called by the NSW Trades and Labour Council to
  protest changes to the state's workers' compensation laws.
      The shipping sources said the various port unions appear to
  be taking it in turn to work for a short time at the start of
  each shift and then to walk off.
      Cargo handling in the port

## NLTKを用いたテキストのトークン抽出, scikit-learnを用いたテキストのTF-IDFモデル抽出

In [3]:
from nltk import word_tokenize
import re # 正規表現
from sklearn.feature_extraction.text import TfidfVectorizer
 
def tokenize(text): # テキストを小文字英語トークンに変換してリストで返す関数
    min_length = 3 # 3文字以上のものだけ残す
    words = [word.lower() for word in word_tokenize(text)]
    p = re.compile('[a-zA-Z]+') # 小文字化しているが一応アルファベットで開始
    filtered_tokens = [token for token in words \
                       if p.match(token) and len(token) >= min_length]
    return filtered_tokens

# TF-IDF重みでベクトル化
vectorizer = TfidfVectorizer(stop_words='english', tokenizer=tokenize)

# 訓練データはfit_transform関数で決められた語彙に基づきTF-IDFを計算
vectorised_train_documents = vectorizer.fit_transform(train_docs)

# テストデータはtransform関数で決められた語彙に基づきTF-IDFを計算
vectorised_test_documents = vectorizer.transform(test_docs)

print(f"TF-IFモデルに変換しました")
print(f"訓練データの文書数x次元数：{vectorised_train_documents.shape}")
print(f"テストデータの文書数x次元数：{vectorised_test_documents.shape}")

TF-IFモデルに変換しました
訓練データの文書数x次元数：(7713, 26978)
テストデータの文書数x次元数：(2987, 26978)


## SVMによる分類

In [36]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import jaccard_score
from sklearn.metrics import hamming_loss
import numpy as np

mlb = MultiLabelBinarizer()

train_labels = mlb.fit_transform([ma_reuters.categories(doc_id) \
                             for doc_id in train_docs_id])
test_labels = mlb.transform([ma_reuters.categories(doc_id) \
                             for doc_id in test_docs_id])
print(vectorised_train_documents.shape, train_labels.shape)

# マルチクラス　マルチラベル分類器で訓練＋予測
OVR_classifier = OneVsRestClassifier(LinearSVC(random_state=41)) 
OVR_classifier.fit(vectorised_train_documents, train_labels)

OVR_predictions = OVR_classifier.predict(vectorised_test_documents)

# Jaccard係数の計算
print (f"Jaccard係数による評価:"
    f"{np.round(jaccard_score(test_labels,OVR_predictions, average='samples'),3)}")

print("カテゴリごとのjaccard_scoreの算出")
max = [-1  ,""]
min = [1000,""]
for category, jscore in zip(categories, jaccard_score(test_labels,OVR_predictions, average=None)):
    if max[0]<jscore:
        max = [jscore, category]
    if min[0]> jscore:
        min = [jscore, category]

print(f"max: {max[1]} : {max[0]}")
print(f"min: {min[1]} : {min[0]}")

# Hamming損失の計算
print (f"Hamming損失による評価:"
    f"{np.round(hamming_loss(test_labels,OVR_predictions),3)}")

(7713, 26978) (7713, 55)
Jaccard係数による評価:0.86
カテゴリごとのjaccard_scoreの算出
max: earn : 0.9690627843494085
min: lead : 0.0
Hamming損失による評価:0.005


## 課題2

55個のカテゴリーのなかで、Jaccard 係数(資料第 6 回参照)が最も高いカテゴリーと最 も低いカテゴリーが何であったかのべよ。

上記で算出されたように、Jaccardが最大と最小となったカテゴリは以下の通りとなった。

- max: earn : 0.9690627843494085
- min: lead : 0.0

# 課題3

ここでのNLTKのTF-IDFモデル+SVM以外の組み合わせで本マルチラベル問題に対応できる手法を適宜ためし、実行結果をのべよ。たとえば、テキスト表現に授業の資料やビデオで述べたWord2VecやBERT等を用いてもよい。NLTKのBoW(TF-IDF)モデルのまま場合、少なくとも分類器は変更すること。たとえば、ニューラルネット(NN)を用いてもよい。NN の場合、評価手法はバイナリークロスエントロピー等の誤差がマルチラベル・マルチクラス分類ではよく用いられる。

今回は今までにずっと使ってみたかったWord2vecを使って分類してみる。

使うモジュールはGensimを使用しており、以下のサイトを参考にしている。

[gensim](https://radimrehurek.com/gensim/models/word2vec.html)

また、word2vecの応用版、doc2vecを利用

[doc2vec](https://qiita.com/naotaka1128/items/2c4551abfd40e43b0146)

In [20]:
from gensim import models

def split_docs(docs: list):
    return [list(filter(None, re.split('[ \n]',td))) for td in docs]


# 参考記事： http://qiita.com/okappy/items/32a7ba7eddf8203c9fa1
class LabeledListSentence(object):
    def __init__(self, words_list, labels):
        self.words_list = words_list
        self.labels = labels

    def __iter__(self):
        for i, words in enumerate(self.words_list):
            yield models.doc2vec.TaggedDocument(words, [self.labels[i]])


In [40]:
splited_train_docs = split_docs(train_docs)
splited_test_docs = split_docs(test_docs)

train_labels_text = [ma_reuters.categories(doc_id) for doc_id in train_docs_id]
print(len(train_labels_text), len(splited_train_docs))

train_sentences = [models.doc2vec.TaggedDocument(words, label) for words,label in zip(splited_train_docs, train_labels_text)]


model = models.Doc2Vec(documents=train_sentences,vector_size=1000, window=2, min_count=1, workers=4)


model.save('doc2vec.model')



7713 7713


## カテゴリごとに素性ベクトルを計算

カテゴリ別に文章を素性ベクトルに変換（単語ごとに特徴量を加算）する

In [41]:
# 学習後はモデルをファイルからロード可能
model = models.Doc2Vec.load('doc2vec.model')

train_vec_list = []
for docs in splited_train_docs:
    train_vec_list.append(model.infer_vector(docs))
train_vec = np.array(train_vec_list)

test_vec_list = []
for docs in splited_test_docs:
    test_vec_list.append(model.infer_vector(docs))
test_vec = np.array(test_vec_list)



# # マルチクラス　マルチラベル分類器で訓練＋予測
OVR_classifier = OneVsRestClassifier(LinearSVC(random_state=41)) 
OVR_classifier.fit(train_vec, train_labels)

OVR_predictions = OVR_classifier.predict(test_vec)

# Jaccard係数の計算
print (f"Jaccard係数による評価:"
    f"{np.round(jaccard_score(test_labels,OVR_predictions, average='samples'),3)}")

/Users/kinouchitakahiro/anaconda3/envs/test_sagemaker/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kinouchitakahiro/anaconda3/envs/test_sagemaker/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kinouchitakahiro/anaconda3/envs/test_sagemaker/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kinouchitakahiro/anaconda3/envs/test_sagemaker/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kinouchitakahiro/anaconda3/envs/test_sagemaker/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to co

Jaccard係数による評価:0.691


## 結果と考察

Doc2vecを用いて他クラスラベリングのタスクを行った。
はじめはDoc2vecの次元数を100程度で行ったが、SVMが収束しなかった.
Jaccard係数による評価:0.689　となり、不十分なベクトル表現であると考えられる。


そこで、word2vecの次元数を2000まで増やすことで、他クラス分類の精度を向上させることができた。
Jaccard係数による評価:0.691

計算時間は非常にかかったがJaccard係数の結果はTF-IDFよりも低い結果となってしまった。この原因として、データセットの単語数が十分ではなく、単語の分散表現がうまく学習できていないためと考えられる。
今回はSVMを使って分類したが、この分類までNNに任せると、より柔軟に分類されると考えられる。